In [ ]:
import pandas as pd

In [ ]:
user_table = pd.read_csv('user_table.csv')
home_page = pd.read_csv('home_page_table.csv')
search_page = pd.read_csv('search_page_table.csv')
payment_page = pd.read_csv('payment_page_table.csv')
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')

# Меняем название одинаковой колонки в каждом датасете
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

# Склеиваем последовательно все датасеты с помощью метода merge по идентификатору user_id
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')

# Для удобства последующих расчетов, перекодируем названия событий в 0/1 с помощью лямбда-функции
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


# 1. построить сегментированную воронку конверсии пользователей по полу

In [ ]:
# Подготовим группировку данных по совершению событий на сайте в разрезе полов
dfg = df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().unstack('sex').unstack('sex').reset_index()

# Уберем ненужный заголовок верхнего уровня
dfg.columns.name = None

# Переименуем стандартные после процедуры заголовки
dfg = dfg.rename(columns = {'index':'Page'})

# Посмотрим, что получилось
dfg.head()

,Page,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [ ]:
# Импортируем графическую библиотеку Plotly 
from plotly import graph_objects as go

# Настраиваем скрипт библиотеки Plotly нашими параметрами
fig = go.Figure()

# Часть графика для Male
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Male'],
    name = 'Male',
    constraintext='outside',
    textposition = 'inside',
    textinfo = 'value+percent previous'
    ))

# Часть графика для Female
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Female'],
    name = 'Female',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

fig.update_layout(
    title = 'Воронка конверсии по полу',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

fig.show()

# 2. Создать 2 новые фичи на основе колонки “date”: месяц и день недели

In [ ]:
df['month'] = pd.DatetimeIndex(df['date']).month
df['day_of_week'] = pd.DatetimeIndex(df['date']).dayofweek
df.head(15)


,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month,day_of_week
0,450007,2015-02-28,Desktop,Female,1,0,0,0,2,5
1,756838,2015-01-13,Desktop,Male,1,0,0,0,1,1
2,568983,2015-04-09,Desktop,Male,1,1,0,0,4,3
3,190794,2015-02-18,Desktop,Female,1,1,0,0,2,2
4,537909,2015-01-15,Desktop,Male,1,0,0,0,1,3
5,993454,2015-03-03,Desktop,Male,1,0,0,0,3,1
6,377150,2015-04-15,Desktop,Female,1,1,0,0,4,2
7,137258,2015-01-27,Mobile,Male,1,0,0,0,1,1
8,608249,2015-04-12,Desktop,Female,1,0,0,0,4,6
9,615615,2015-02-24,Mobile,Female,1,1,0,0,2,1


# 3. Определить самый топовый по продажам месяц и день недели

In [ ]:
df['month'].loc[df['payment_confirmation']==1].value_counts()

1    189
2    173
4     46
3     44
Name: month, dtype: int64

In [ ]:
df['day_of_week'].loc[df['payment_confirmation']==1].value_counts()

0    77
5    70
1    69
3    66
6    62
4    54
2    54
Name: day_of_week, dtype: int64

# 4. Выяснить, в какой день недели лучше покупают женщины, а в какой мужчины

In [ ]:
dfg = pd.crosstab(df['day_of_week'], df['sex'] )
dfg.columns.name = None
dfg.head(10)

,Female,Male
day_of_week,,
0,6400,6555
1,6410,6287
2,6359,6472
3,6728,6716
4,6328,6422
5,6344,6457
6,6506,6416


In [ ]:
dfg.sort_values(by='Female', ascending = False).head(1)

,Female,Male
day_of_week,,
3,6728,6716


In [ ]:
dfg.sort_values(by='Male', ascending = False).head(1)

,Female,Male
day_of_week,,
3,6728,6716


In [ ]:
dfg.max() # для проверки себя)

Female    6728
Male      6716
dtype: int64